In [1]:
def get_input(fname="input.txt"):
    with open(fname) as f:
        return [int(n) for n in f.readline().strip().split(",")]

In [2]:
test_instructions = get_input("test.txt")

In [3]:
test_instructions[:5]

[3, 23, 3, 24, 1002]

In [4]:
test_instructions[-5:]

[4, 23, 99, 0, 0]

In [5]:
def run(instructions, inp):
    instructions = instructions[:]
    outputs = []
    idx = 0
    while True:
        opcode = instructions[idx]
        op = opcode % 100
        opcode //= 100
        if op == 99: # halt
            break
        elif op in (1, 2, 7, 8): # addition, multiplication, less than, equals
            modes = []
            for i in range(3):
                modes.append(opcode % 10)
                opcode //= 10
            idx_a, idx_b, idx_o = idx + 1, idx + 2, idx + 3
            mode_a, mode_b, mode_o = modes
            a = instructions[idx_a]
            if mode_a == 0:
                a = instructions[a]
            b = instructions[idx_b]
            if mode_b == 0:
                b = instructions[b]
            o = instructions[idx_o]
            if op in (1, 2): # addition, multiplication
                res = a + b if op == 1 else a * b
            elif op == 7:
                res = int(a < b)
            elif op == 8:
                res = int(a == b)
            instructions[o] = res
            idx += 4
        elif op in (5, 6): # jump if true / false
            modes = []
            for i in range(2):
                modes.append(opcode % 10)
                opcode //= 10
            idx_a, idx_b = idx + 1, idx + 2
            mode_a, mode_b = modes
            a = instructions[idx_a]
            if mode_a == 0:
                a = instructions[a]
            b = instructions[idx_b]
            if mode_b == 0:
                b = instructions[b]
            if op == 5:
                if a != 0:
                    idx = b
                else:
                    idx += 3
            elif op == 6:
                if a == 0:
                    idx = b
                else:
                    idx += 3
        elif op == 3: # read input
            instructions[instructions[idx + 1]] = inp.pop()
            idx += 2
        elif op == 4: # write output
            outputs.append(instructions[instructions[idx + 1]])
            idx += 2
    return outputs

In [6]:
run(test_instructions, [0, 0])

[5]

In [7]:
test_phases = [0,1,2,3,4]
test_input = 0

In [8]:
for phase in test_phases:
    result = run(test_instructions, [test_input, phase])
    test_input = result[-1]

In [9]:
test_input

54321

In [10]:
actual_input = get_input()

In [11]:
from itertools import permutations 

In [12]:
phase_combinations = list(permutations(range(5)))

In [13]:
phase_combinations[0]

(0, 1, 2, 3, 4)

In [14]:
phase_combinations[-1]

(4, 3, 2, 1, 0)

In [15]:
max_output = 0

In [16]:
for phases in phase_combinations:
    output = 0
    for phase in phases:
        result = run(actual_input, [output, phase])
        output = result[-1]
    max_output = max(max_output, output)

In [17]:
max_output

255840

In [18]:
import itertools

In [19]:
test_input = get_input("test2.txt")

In [20]:
class Amplifier(object):
    instructions = None
    input = None
    output = None
    outputs = None
    is_halted = False
    idx = 0
    
    STOPPED = -1
    NEEDS_INPUT = -2
    
    def __init__(self, instructions, input, output):
        self.instructions = instructions[:]
        self.input = input
        self.output = output
        self.outputs = []
    
    def step(self):
        if self.is_halted:
            return Amplifier.STOPPED
        opcode = self.instructions[self.idx]
        op = opcode % 100
        opcode //= 100
        if op == 99: # halt
            self.is_halted = True
            return Amplifier.STOPPED
        elif op in (1, 2, 7, 8): # addition, multiplication, less than, equals
            modes = []
            for i in range(3):
                modes.append(opcode % 10)
                opcode //= 10
            idx_a, idx_b, idx_o = self.idx + 1, self.idx + 2, self.idx + 3
            mode_a, mode_b, mode_o = modes
            a = self.instructions[idx_a]
            if mode_a == 0:
                a = self.instructions[a]
            b = self.instructions[idx_b]
            if mode_b == 0:
                b = self.instructions[b]
            o = self.instructions[idx_o]
            if op in (1, 2): # addition, multiplication
                res = a + b if op == 1 else a * b
            elif op == 7:
                res = int(a < b)
            elif op == 8:
                res = int(a == b)
            self.instructions[o] = res
            self.idx += 4
        elif op in (5, 6): # jump if true / false
            modes = []
            for i in range(2):
                modes.append(opcode % 10)
                opcode //= 10
            idx_a, idx_b = self.idx + 1, self.idx + 2
            mode_a, mode_b = modes
            a = self.instructions[idx_a]
            if mode_a == 0:
                a = self.instructions[a]
            b = self.instructions[idx_b]
            if mode_b == 0:
                b = self.instructions[b]
            if op == 5:
                if a != 0:
                    self.idx = b
                else:
                    self.idx += 3
            elif op == 6:
                if a == 0:
                    self.idx = b
                else:
                    self.idx += 3
        elif op == 3: # read input
            if len(self.input) == 0:
                return Amplifier.NEEDS_INPUT
            self.instructions[self.instructions[self.idx + 1]] = self.input.popleft()
            self.idx += 2
        elif op == 4: # write output
            output = self.instructions[self.instructions[self.idx + 1]]
            self.outputs.append(output)
            self.output.append(output)
            self.idx += 2
    

In [21]:
from collections import deque

In [22]:
phases = (9,7,8,5,6)
queues = [deque([i]) for i in phases]
queues[0].append(0)

amplifiers = deque([])
for i in range(5):
    amp = Amplifier(test_input, queues[i % 5], queues[(i + 1) % 5])
    amplifiers.append(amp)

In [23]:
while not all(amp.is_halted for amp in amplifiers):
    amp = amplifiers.popleft()
    amplifiers.append(amp)
    while amp.step() not in (Amplifier.STOPPED, Amplifier.NEEDS_INPUT): pass
#     camp = 
#     while amplifiers[0]

In [24]:
queues[0]

deque([18216])

In [25]:
test_input = get_input("test3.txt")

In [26]:
phases = (9,8,7,6,5)
queues = [deque([i]) for i in phases]
queues[0].append(0)

amplifiers = deque([])
for i in range(5):
    amp = Amplifier(test_input, queues[i % 5], queues[(i + 1) % 5])
    amplifiers.append(amp)

In [27]:
while not all(amp.is_halted for amp in amplifiers):
    amp = amplifiers.popleft()
    amplifiers.append(amp)
    while amp.step() not in (Amplifier.STOPPED, Amplifier.NEEDS_INPUT): pass

In [28]:
queues[0]

deque([139629729])

In [29]:
phase_combinations = list(permutations(range(5, 10)))

In [30]:
max_output = 0

In [31]:
for phases in phase_combinations:
    queues = [deque([i]) for i in phases]
    queues[0].append(0)

    amplifiers = deque([])
    for i in range(5):
        amp = Amplifier(actual_input, queues[i % 5], queues[(i + 1) % 5])
        amplifiers.append(amp)
    while not all(amp.is_halted for amp in amplifiers):
        amp = amplifiers.popleft()
        amplifiers.append(amp)
        while amp.step() not in (Amplifier.STOPPED, Amplifier.NEEDS_INPUT): pass
    max_output = max(max_output, queues[0][-1])

In [32]:
max_output

84088865